In [59]:
import json
import elasticsearch
from dotenv import load_dotenv
import os

load_dotenv()

True

In [60]:
# openai api
client = OpenAI()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [7]:
base_folder = 'D:/Projects/AI-Restaurent-Chat-bot/'
input_data_folder = base_folder+'input_data/'

In [8]:
with open(input_data_folder + 'food_user_qa_dataset.json', 'rt') as f_in:
    data = json.load(f_in)

In [10]:
documents = []
for dish in data['dishes']:
    dish_name = dish['dish name']
    for doc in dish['documents']:
        doc['dish_name'] = dish_name  # Add dish_name to each document
        documents.append(doc)

In [11]:
documents[question]

NameError: name 'question' is not defined

In [12]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '27bfd72e4b73', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Tx4GlF-MTPeTcrgVgX2HGw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [13]:
# Creating an index, adjusted for the dish data
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "dish_name": {"type": "keyword"} , # Updated from 'course' to 'dish_name'
            "id": {"type": "keyword"}
        }
    }
}



In [14]:
# Create index
index_name = "menu-items"
response = es.indices.create(index=index_name, body=index_settings)
print("Index creation response:", response)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [menu-items/FCOSRjC0SOyQTQ0x4841SA] already exists')

In [17]:
from tqdm.auto import tqdm
# Index documents
for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

  0%|          | 0/8720 [00:00<?, ?it/s]

BadRequestError: BadRequestError(400, 'mapper_parsing_exception', 'failed to parse')

In [19]:
query = "What is the cost of banana cake"

In [23]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "category": "menu items"  # Filter by category
                    }
                }
            }
        }
    }

    # Perform the search
    response = es.search(index=index_name, body=search_query)

    result_doc =[]

    for hit in response['hits']['hits']:
        result_doc = hit['_source']
        print(f"Section: {doc['section']}")
        print(f"Question: {doc['question']}")
        print(f"Answer: {doc['text']}")
        print("\n")

    return result_doc

In [24]:
elastic_search(query)

[]

In [53]:
def build_prompt(user_question, search_results):
    prompt_template = """You are a highly trained professional who have good knowledge on the menu. Answer the questions which user asks based on the  CONTEXT from teh menu dataset. 
    Use only factsfrom the CONTEXT when answering the QUESTION. If the CONTEXT doesnt contain the amswer, output generic answer'
    QUESTION :{question}
    CONTEXT : {context}
    """.strip()
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [54]:
from openai import OpenAI
client = OpenAI()


def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [57]:
query = 'cost of jacks fish'

def rag(query):
    search_results = elastic_search(query)
    prompt=build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [58]:
rag(query)


"I'm sorry, but without the specific menu information, I am unable to provide the cost of Jack's Fish. It would be best to refer to the menu directly."